In [1]:
import streamlit as st
import pandas as pd
import plotly.express as px
from lifetimes import BetaGeoFitter, GammaGammaFitter
from lifetimes.utils import summary_data_from_transaction_data

st.set_page_config(layout="wide")
st.title("Customer Lifetime Value (CLV) Calculator for MOEVE")

# Upload CSV file
uploaded_file = st.file_uploader("Upload a CSV file", type=["csv"])

if uploaded_file is not None:
    df = pd.read_csv(uploaded_file)

    # Check required columns
    required_cols = {'CustomerID', 'InvoiceDate', 'Amount'}
    if not required_cols.issubset(df.columns):
        st.error(f"CSV must contain these columns: {required_cols}")
    else:
        df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

        # Compute RFM summary
        summary = summary_data_from_transaction_data(
            df,
            customer_id_col='CustomerID',
            datetime_col='InvoiceDate',
            monetary_value_col='Amount',
            observation_period_end=df['InvoiceDate'].max()
        )

        summary = summary[summary['frequency'] > 0]

        # Fit models
        bgf = BetaGeoFitter(penalizer_coef=0.001)
        bgf.fit(summary['frequency'], summary['recency'], summary['T'])

        ggf = GammaGammaFitter(penalizer_coef=0.01)
        ggf.fit(summary['frequency'], summary['monetary_value'])

        # Predict CLV
        summary['expected_avg_value'] = ggf.conditional_expected_average_profit(
            summary['frequency'], summary['monetary_value']
        )
        summary['clv'] = ggf.customer_lifetime_value(
            bgf,
            summary['frequency'],
            summary['recency'],
            summary['T'],
            summary['monetary_value'],
            time=6,
            freq='D',
            discount_rate=0.01
        )

        # Add percentile and segment
        summary['clv_percentile'] = summary['clv'].rank(pct=True)
        summary['clv_quartile'] = pd.qcut(summary['clv_percentile'], q=4, labels=[
            'Q1 - Low', 'Q2 - Mid-Low', 'Q3 - Mid-High', 'Q4 - Top'])

        st.success("CLV computation completed!")

        # Donut chart with Plotly
        fig = px.pie(summary, names='clv_quartile', hole=0.5, title="CLV Distribution by Quartile")
        st.plotly_chart(fig, use_container_width=True)

        # Slider to filter CLV range
        clv_min, clv_max = float(summary['clv'].min()), float(summary['clv'].max())
        clv_range = st.slider("Filter customers by CLV range", clv_min, clv_max, (clv_min, clv_max))
        filtered_summary = summary[(summary['clv'] >= clv_range[0]) & (summary['clv'] <= clv_range[1])]

        # Show filtered table
        st.dataframe(filtered_summary[['frequency', 'recency', 'T', 'monetary_value', 'clv', 'clv_quartile']])

        # Download button
        csv = filtered_summary.to_csv(index=True).encode('utf-8')
        st.download_button(
            label="Download Filtered Results as CSV",
            data=csv,
            file_name='clv_filtered_results.csv',
            mime='text/csv'
        )


2025-05-05 13:37:26.702 
  command:

    streamlit run C:\Users\HP\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
